In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
from IPython.core.display import display, HTML
import pandas as pd
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',100)    

display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from itertools import combinations, takewhile
import collections
import pandas as pd
import numpy as np
import os

from simhash import Simhash, SimhashIndex

sns.set()

print(sys.version)

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]


In [5]:
fpath = os.path.join(os.getcwd(), "temp.csv")
tweets_df = pd.read_csv(fpath, index_col="Unnamed: 0")
display(
    tweets_df.shape,
    tweets_df.head()
)

(1048947, 20)

,id_str,text,in_reply_to_status_id_str,in_reply_to_user_id_str,created_at,user_id_str,user_name,user_followers_count,user_favourites_count,user_statuses_count,user_friends_count,coordinates,favorite_count,entities_hashtags,favorited,retweet_count,retweeted,retweeted_status_user_id_str,retweeted_status_user_name,hashtags_cleaned
0,8.778927e+17,RT @ArkansasBlog: Study: States with concealed carry laws experience rise in violent crime. http...,NaN,NaN,Thu Jun 22 14:15:01 +0000 2017,1.550783e+08,Lake Man,1554.0,56574.0,95959.0,2577.0,NaN,0.0,[],False,0.0,False,17131180,Arkansas Blog,[]
1,8.778927e+17,"IL Healthcare Truth Tour: Heal Don't Repeal! 6/22 2:30PM Federal Plaza, Chicago. Mobile billboar...",NaN,NaN,Thu Jun 22 14:15:01 +0000 2017,7.159992e+06,Annie Williams,22.0,167.0,417.0,87.0,NaN,0.0,[],False,0.0,False,NaN,NaN,[]
2,8.778927e+17,"IL Healthcare Truth Tour: Heal Don't Repeal! 6/22 2:30PM Federal Plaza, Chicago. Mobile billboar...",NaN,NaN,Thu Jun 22 14:15:01 +0000 2017,8.271887e+17,Indivisible9IL,264.0,111.0,342.0,231.0,NaN,0.0,[],False,0.0,False,NaN,NaN,[]
3,8.778927e+17,"IL Healthcare Truth Tour: Heal Don't Repeal! 6/22 2:30PM Federal Plaza, Chicago. Mobile billboar...",NaN,NaN,Thu Jun 22 14:15:02 +0000 2017,8.313191e+17,Indivisible Illinois,1027.0,1626.0,982.0,267.0,NaN,0.0,[],False,0.0,False,NaN,NaN,[]
4,8.778927e+17,Day 1 of Chicago Summit: Gregory Touhill delivers keynote on Cybersecurity=Risk Management #ISMG...,NaN,NaN,Thu Jun 22 14:15:02 +0000 2017,3.998460e+07,ISMG Network News,2395.0,119.0,12833.0,960.0,NaN,0.0,"[Row(indices=[91, 103], text='ISMGSummits'), Row(indices=[104, 112], text='infosec')]",False,0.0,False,NaN,NaN,"['ISMGSummits', 'infosec']"


# Q1 - Identify tweets related to UChicago and 3-4 universities of your choice 

In [6]:
uc_favored_tags = ["uchicago", "uchearing", "universityofchicago", "pritzkerschoolofmedicine",
                   "uofc", "maroonmade", "chicagobooth"]

upenn_favored_tags = ["upenn", "penn", "uofpenn", "universityofpennsylvania", "pennlaw"]

uic_favored_tags = ["UIC", "UICProud"]

In [7]:
t = ['uchicago', 'firstgen', 'college', 'careers', 'trioprograms']
for ftag in uc_favored_tags:
    for tag in t:
        if ftag.lower() in tag.lower():
            print(tag)

uchicago


In [8]:
from ast import literal_eval

def le(row):
    if row is np.NaN:
        return []
    else:
        return literal_eval(row)

tweets_df['hashtags_cleaned']=tweets_df['hashtags_cleaned'].apply(le)

In [9]:
uc_favored_tags = ["uchicago", "uchearing", "universityofchicago", "pritzkerschoolofmedicine",
                   "uofc", "maroonmade", "chicagobooth"]

def is_uc_tweet(row):
    if row is np.NaN:
        return False
    for ftag in uc_favored_tags:
        for tag in row:
            if ftag.lower() in tag.lower():
                return True
    return False

tweets_df["uc_tweet"] = tweets_df["hashtags_cleaned"].apply(is_uc_tweet)

In [10]:
uic_favored_tags = ["UIC", "UICProud"]
def is_uic_tweet(row):
    if row is np.NaN:
        return False
    for ftag in uic_favored_tags:
        for tag in row:
            if ftag.lower() == tag.lower():
                return True
    return False

tweets_df["uic_tweet"] = tweets_df["hashtags_cleaned"].apply(is_uic_tweet)

In [11]:
upenn_favored_tags = ["upenn", "penn", "uofpenn", "universityofpennsylvania", "pennlaw"]

def is_upenn_tweet(row):
    if row is np.NaN:
        return False
    for ftag in upenn_favored_tags:
        for tag in row:
            if ftag.lower() in tag.lower():
                return True
    return False

tweets_df["upenn_tweet"] = tweets_df["hashtags_cleaned"].apply(is_upenn_tweet)

In [12]:
mit_favored_tags = ["MIT"]

def is_mit_tweet(row):
    if row is np.NaN:
        return False
    for ftag in mit_favored_tags:
        for tag in row:
            if ftag.lower() == tag.lower():
                return True
    return False

tweets_df["mit_tweet"] = tweets_df["hashtags_cleaned"].apply(is_mit_tweet)

In [23]:
nyu_favored_tags = ["nyu"]

def is_nyu_tweet(row):
    if row is np.NaN:
        return False
    for ftag in nyu_favored_tags:
        for tag in row:
            if ftag.lower() in tag.lower():
                return True
    return False

tweets_df["nyu_tweet"] = tweets_df["hashtags_cleaned"].apply(is_nyu_tweet)

In [22]:
stanford_favored_tags = ["stanford"]

def is_stanford_tweet(row):
    if row is np.NaN:
        return False
    for ftag in stanford_favored_tags:
        for tag in row:
            if ftag.lower() in tag.lower():
                return True
    return False

tweets_df["stanford_tweet"] = tweets_df["hashtags_cleaned"].apply(is_stanford_tweet)

In [27]:
nw_favored_tags = ["northwestern"]

def is_nw_tweet(row):
    if row is np.NaN:
        return False
    for ftag in nw_favored_tags:
        for tag in row:
            if ftag.lower() == tag.lower():
                return True
    return False

tweets_df["nw_tweet"] = tweets_df["hashtags_cleaned"].apply(is_nw_tweet)

In [29]:
display(
    tweets_df[tweets_df["uic_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df[tweets_df["uc_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df[tweets_df["upenn_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
#     tweets_df[tweets_df["mit_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
#     tweets_df[tweets_df["nyu_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df[tweets_df["nw_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
#     tweets_df[tweets_df["stanford_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df.shape
)

(133, 3)

(93, 3)

(88, 3)

(214, 3)

(1048947, 27)

------------------------

### Universities to monitor

1. University of Chicago
1. University of Illinois, Chicago
1. University of Pennsylvania
1. Northwestern University

In [34]:
uc_favored_tags = ["uchicago", "uchearing", "universityofchicago", "pritzkerschoolofmedicine",
                   "uofc", "maroonmade", "chicagobooth"]
nw_favored_tags = ["northwestern"]
upenn_favored_tags = ["upenn", "penn", "uofpenn", "universityofpennsylvania", "pennlaw"]
uic_favored_tags = ["UIC", "UICProud"]

def is_upenn_tweet(row):
    if row is np.NaN:
        return False
    for ftag in upenn_favored_tags:
        for tag in row:
            if ftag.lower() in tag.lower():
                return True
    return False

def is_nw_tweet(row):
    if row is np.NaN:
        return False
    for ftag in nw_favored_tags:
        for tag in row:
            if ftag.lower() == tag.lower():
                return True
    return False

def is_uc_tweet(row):
    if row is np.NaN:
        return False
    for ftag in uc_favored_tags:
        for tag in row:
            if ftag.lower() in tag.lower():
                return True
    return False

def is_uic_tweet(row):
    if row is np.NaN:
        return False
    for ftag in uic_favored_tags:
        for tag in row:
            if ftag.lower() == tag.lower():
                return True
    return False

tweets_df["nw_tweet"] = tweets_df["hashtags_cleaned"].apply(is_nw_tweet)
tweets_df["upenn_tweet"] = tweets_df["hashtags_cleaned"].apply(is_upenn_tweet)
tweets_df["uc_tweet"] = tweets_df["hashtags_cleaned"].apply(is_uc_tweet)
tweets_df["uic_tweet"] = tweets_df["hashtags_cleaned"].apply(is_uic_tweet)

display(
    tweets_df[tweets_df["uic_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df[tweets_df["uc_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df[tweets_df["upenn_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df[tweets_df["nw_tweet"]==True][["id_str", "text", "hashtags_cleaned"]].shape,
    tweets_df.shape
)

(133, 3)

(93, 3)

(88, 3)

(214, 3)

(1048947, 27)